# EMR Notebook SageMaker Custom Abalone Ring Estimator

1. [Setup](#Setup)
2. [Load the Data](#Load-the-Data)
3. [Train the Model](#Train-the-Model)
4. [Inference Results](#Inference-Results)
5. [Wrap-Up](#Wrap-Up)

## Setup

In [1]:
# *****DEFINE USER SPECIFIC PARAMETERS******
region = 'us-east-1'
sagemaker_execution_role = 'arn:aws:iam::620614497509:role/SagemakerExecutionRole'

#The number of EMR nodes to process the data.
num_workers = 12

# The location of the dataset we will be using. 
source_bucket = 'ee-assets-prod-us-east-1'
source_key = 'modules/8560b4bd6942403b9fe3291928df2453/v1/data/abalone.csv'

if (region and source_bucket and sagemaker_execution_role and num_workers):
    print('All necessary user parameters are entered.')
else:
    print('Please check to make sure you entered all default parameters!')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1646957004110_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

All necessary user parameters are entered.

Each EMR notebook is launched with its own Spark context (variable sc). A Spark Context is the entry point for communication with Spark. First you need to install the Python packages that you'll use throughout the notebook. EMR notebooks come with a default set of libraries for data processing. You can see which libraries are installed on the notebook by calling the Spark Context's list_packages() function. 

In [2]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aws-cfn-bootstrap (2.0)
beautifulsoup4 (4.9.3)
boto (2.49.0)
click (8.0.1)
docutils (0.14)
jmespath (0.10.0)
joblib (1.0.1)
lockfile (0.11.0)
lxml (4.6.3)
mysqlclient (1.4.2)
nltk (3.6.2)
nose (1.3.4)
numpy (1.16.5)
pip (9.0.1)
py-dateutil (2.2)
pystache (0.5.4)
python-daemon (2.2.3)
python37-sagemaker-pyspark (1.4.1)
pytz (2021.1)
PyYAML (5.4.1)
regex (2021.8.3)
setuptools (28.8.0)
simplejson (3.2.0)
six (1.13.0)
tqdm (4.62.1)
wheel (0.29.0)
windmill (1.6)

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

To comunicate with SageMaker you need to install notebook scoped libraries. These libraries are available only during the notebook session. After the session ends, the libraries are deleted. 

We install [boto3 (the AWS Python 3 SDK)](https://aws.amazon.com/sdk-for-python/) and the [high level SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/). 

In [3]:
sc.install_pypi_package("boto3==1.16.9");
sc.install_pypi_package('sagemaker==2.16.1');

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/e3/99/3979b617c0cbb3d7260cd3357b4a06edaa91073dd252687b7502f6678bb8/boto3-1.16.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2c/05/0a955f0c92bec7da076fbbc73926dfb13fab8e2b88de7f8eb17c443f28f0/botocore-1.19.63-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/00/89/0cb4e92c239e6425b9b0035227b8cdf9d3d098a5c9e95632c3815df63a09/s3transfer-0.3.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4e/b8/f5a25b22e803f0578e668daa33ba3701bb37858ec80e08a150bd7d2cf1b1/urllib3-1.26.8-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/92/f2/c48787ca7d1e20daa185e1b6b2d4e16acd2fb5e0320bc50ffc89b91fa4d7/importlib_metadata-4.11.3-py3-none-any.whl
  Using cached https://files.python

In [4]:
import boto3
import sagemaker

boto_sess = boto3.Session(region_name=region)
sage_sdk_session = sagemaker.Session(boto_session=boto_sess)
bucket = sage_sdk_session.default_bucket()

print('A SageMaker session was initiated! You are using {} as your S3 bucket for intermediate files.'.format(bucket))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

A SageMaker session was initiated! You are using sagemaker-us-east-1-620614497509 as your S3 bucket for intermediate files.

## Load the Data

We will use the public abalone data set from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Abalone)
to train and test a regression model.

   Given in the dataset is the attribute name, attribute type, the measurement unit and a
   brief description.  The number of rings is the value to predict: either
   as a continuous value or as a classification problem.
   
   The age of an abalone is the number of rings in the shell + 1.5 years. Without a model researchers must cut through the abalone shell
   and use a microscope to count the rings. Using a model to predict rings eliminates this time consuming process.

	Name			Data Type		Meas.	Description
	----			---------		-----	-----------
	Rings			integer					+1.5 gives the age in years
	Length			continuous		mm		Longest shell measurement
	Diameter		continuous		mm		perpendicular to length
	Height			continuous		mm		with meat in shell
	Whole weight	continuous		grams	whole abalone
	Shucked weight	continuous		grams	weight of meat
	Viscera weight	continuous		grams	gut weight (after bleeding)
	Shell weight	continuous		grams	after being dried
	Male			integer			1/0 	1 encodes true, 0 false
	Female			integer			1/0 	1 encodes true, 0 false
	Infant			integer			1/0 	1 encodes true, 0 false

First, we need to copy the public files to the S3 bucket in our account.

In [5]:
s3 = boto3.client('s3', region_name=region)

# Local bucket S3 prefix to store the data under.
local_key = 'data/abalone.csv'

s3.copy(CopySource={'Bucket' : source_bucket,
                    'Key' : source_key}, 
        Bucket=bucket, 
        Key=local_key)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Read the dataset from S3 in to a Spark dataframe.
abalone_data = spark.read.load(f's3://{bucket}/{local_key}', format='csv', inferSchema=True, header=True).repartition(num_workers)
abalone_data.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+--------+------+------------+--------------+--------------+------------+----+------+------+
|Rings|Length|Diameter|Height|Whole_weight|Shucked_weight|Viscera_weight|Shell_weight|Male|Female|Infant|
+-----+------+--------+------+------------+--------------+--------------+------------+----+------+------+
|    9| 0.665|   0.505| 0.165|       1.349|        0.5985|        0.3175|        0.36|   0|     1|     0|
|    7| 0.505|    0.39| 0.185|      0.6125|         0.267|         0.142|       0.172|   0|     0|     1|
|   13| 0.575|   0.475|  0.17|       0.967|        0.3775|         0.284|       0.275|   0|     0|     1|
|    9| 0.455|   0.355| 0.105|       0.372|         0.138|        0.0765|       0.135|   0|     0|     1|
|   17|  0.52|   0.425| 0.155|      0.7735|         0.297|         0.123|       0.255|   1|     0|     0|
+-----+------+--------+------+------------+--------------+--------------+------------+----+------+------+
only showing top 5 rows

Now that the data is in Spark we can modify and enhance our data. As an example, including all four abalone weights may be unnecessary. What really matters may be the difference between the whole weight and the shell weight. Making such changes on large datasets can be done easily in Spark.

Let's try adding a column that is the difference between whole weight and shell weight. Then remove the whole, shucked, weight, and shell weight columns. 

In [7]:
abalone_data = abalone_data.withColumn('Difference_weight', abalone_data.Whole_weight - abalone_data.Shell_weight)
abalone_data = abalone_data.drop('Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight')
abalone_data.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+--------+------+----+------+------+------------------+
|Rings|Length|Diameter|Height|Male|Female|Infant| Difference_weight|
+-----+------+--------+------+----+------+------+------------------+
|   16|  0.74|     0.6| 0.195|   0|     1|     0|             1.264|
|   13| 0.655|    0.52| 0.175|   0|     1|     0|             1.022|
|    6| 0.385|    0.29| 0.095|   0|     0|     1|             0.226|
|   23|   0.8|    0.63| 0.195|   0|     1|     0|1.9059999999999997|
|   11|  0.63|    0.48|  0.16|   0|     1|     0|0.8840000000000001|
+-----+------+--------+------+----+------+------+------------------+
only showing top 5 rows

In [8]:
# Split the dataframe in to training and validation data.
# The training will be used to refine our model.
# The test data will be used to measure the model's accuracy.
train_data, test_data = abalone_data.randomSplit([.75,.25])

s3_train = f's3://{bucket}/train/'
s3_test = f's3://{bucket}/test/'
data_format = 'csv'

# Save the data in to S3 for training by SageMaker
train_data.write.save(s3_train, format=data_format, mode='overwrite')
test_data.write.save(s3_test, format=data_format, mode='overwrite')

print(f'Training dataset saved in {data_format} format to {s3_train}!')
print(f'Testing dataset saved in {data_format} format to {s3_test}!')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training dataset saved in csv format to s3://sagemaker-us-east-1-620614497509/train/!
Testing dataset saved in csv format to s3://sagemaker-us-east-1-620614497509/test/!

## Train the Model
SageMaker contains several common built-in algorithms. For this lab you have the choice of using either the [LinearLearner](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html) or [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) built-in algorithms. Both are regression models that estimate the number of rings on the abalone.

In [9]:
# Uncomment the LinearLearner line to use the LinearLearner algorithm. 
model = 'xgboost'
#model = 'linear-learner'

print('The SageMaker {} model will be used.'.format(model))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The SageMaker xgboost model will be used.

In [10]:
# Set the regularization weights. Increasing these will reduce how closely the model fits to the training data.
l1 = .25
l2 = .25

# Hyperparameters for XGBoost algorithm
xgboost_params = {
    'num_round':100,
    'objective': 'reg:linear',
    'alpha': l1,
    'lambda': l2
}

# Hyperparameters for LinearLearner algorithm
linear_params = {
    'feature_dim':len(abalone_data.columns)-1,
    'predictor_type': 'regressor',
    'loss': 'squared_loss',
    'l1': l1,
    'wd': l2
}

hyperparams = {
    'linear-learner': linear_params,
    'xgboost': xgboost_params
}

print('All model parameters have been set!')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

All model parameters have been set!

In [11]:
from sagemaker.image_uris import retrieve

estimator = sagemaker.estimator.Estimator(
    image_uri=retrieve(framework=model, region=region, version='latest', py_version='py3'), 
    role=sagemaker_execution_role, 
    train_instance_count=1, 
    train_instance_type='ml.m5.large',
    sagemaker_session=sage_sdk_session, 
    hyperparameters=hyperparams[model]
)

print('The SageMaker model was constructed with parameters: {}.'.format(estimator.hyperparameters()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The SageMaker model was constructed with parameters: {'num_round': 100, 'objective': 'reg:linear', 'alpha': 0.25, 'lambda': 0.25}.

Now that we initialized the model, we can train the model by calling the fit() function. After calling fit(), SageMaker will create a training instance, train a model on the instance, save the model artifacts to S3, then take down the training instance.

This usually takes about 3 minutes. 

(**Optional**) While you wait, you may check the model training progress through the SageMaker console by following these instructions:  
a.	Open SageMaker console in AWS.  
b.	On the left panel, scroll until you see ‘training jobs’ beneath the ‘Training’ section.  
c.	Click into the job to examine further details; wait until you see the status change to ‘Completed’.


In [19]:
train_channel = sagemaker.session.s3_input(s3_train + 'part', content_type='text/csv')
estimator.fit({'train': train_channel})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2022-03-14 17:19:55 Starting - Starting the training job...
2022-03-14 17:20:22 Starting - Preparing the instances for training.........
2022-03-14 17:21:35 Downloading - Downloading input data...
2022-03-14 17:22:26 Training - Downloading the training image.Arguments: train
[2022-03-14:17:22:29:INFO] Running standalone xgboost training.
[2022-03-14:17:22:29:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-03-14:17:22:29:INFO] File size need to be processed in the node: 0.11mb. Available memory size in the node: 320.92mb
[2022-03-14:17:22:29:INFO] Determined delimiter of CSV input is ','
[17:22:29] S3DistributionType set as FullyReplicated
[17:22:29] 3085x7 matrix with 21595 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[17:22:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=6
[0]#011train-rmse:7.19716
[17:22:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra node

## Inference Results

How well did our model perform? Let's see how it does on the test data set we saved to S3 earlier. We'll use [SageMaker batch transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) to run our test data set through our model. Batch transform creates a SageMaker instance, deploys the model, runs the dataset through the model, then takes down the instance. 

In [14]:
s3_inference = s3_train.replace('train', 'inference')

transformer = estimator.transformer(
    instance_count = 1,
    instance_type = 'ml.m5.large',
    strategy = 'MultiRecord',
    output_path = s3_inference,
    assemble_with= 'Line',
    accept=('text/'+data_format)
)

print('SageMaker batch transform initialized with the following parameters:')
for key in transformer.__dict__:
    print('{}:{}'.format(key, transformer.__dict__[key]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SageMaker batch transform initialized with the following parameters:
model_name:xgboost-2022-03-14-17-11-09-634
strategy:MultiRecord
env:None
output_path:s3://sagemaker-us-east-1-620614497509/inference/
output_kms_key:None
accept:text/csv
assemble_with:Line
instance_count:1
instance_type:ml.m5.large
volume_kms_key:None
max_concurrent_transforms:None
max_payload:None
tags:None
base_transform_job_name:xgboost
_current_job_name:None
latest_transform_job:None
_reset_output_path:False
sagemaker_session:<sagemaker.session.Session object at 0x7f74af60ed50>

The transform() function initiates the SageMaker batch transform job. SageMaker will create an inference instance, run the specified test set through the model, save the results to S3, and take down the inference instance. Batch transform is a great option if you require inference for large datasets and don't need sub-second response time.

This usually takes 3 minutes. 

(**Optional**) While you wait, you may check the batch transform progress through the SageMaker console by following these instructions:  
a.	Open SageMaker console in AWS.  
b.	On the left panel, scroll until you see ‘Batch transform jobs’ beneath the ‘Inference’ section.  
c.	Click into the job to examine further details; wait until you see the status change to ‘Completed’.

In [15]:
# The test data set still contains the "Rings" column the model tries to predict. 
# We do not want to send this column to the model, though. We use the SageMaker
# input_filter to filter out that column before sending to the model. We then
# join the model output with the input so we can compare the actual Rings count
# to the predicted count.
transformer.transform(
    data=s3_test,
    content_type='text/csv',
    split_type='Line',
    input_filter='$[1:]',
    join_source='Input',
    wait=True
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

............................Arguments: serve
[2022-03-14 17:15:47 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2022-03-14 17:15:47 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2022-03-14 17:15:47 +0000] [1] [INFO] Using worker: gevent
[2022-03-14 17:15:47 +0000] [21] [INFO] Booting worker with pid: 21
[2022-03-14 17:15:47 +0000] [22] [INFO] Booting worker with pid: 22
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)', 'urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)']. 
  monkey.patch_all(subprocess=True)
[2022-03-14:17

SageMaker batch transform completed and saved the model inference results to S3. Now let's pull the results in to Spark for analysis.

In [16]:
from copy import deepcopy
from pyspark.sql.types import FloatType

# Read the schema from the initial dataset so you can apply it to the inference data.
schema = deepcopy(abalone_data.schema)
schema.add("Estimated_rings", FloatType())

# Pull down the inference data from S3
inference_data = spark.read.load(s3_inference, format=data_format, schema=schema).repartition(num_workers)
inference_data.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+--------+------+----+------+------+-----------------+---------------+
|Rings|Length|Diameter|Height|Male|Female|Infant|Difference_weight|Estimated_rings|
+-----+------+--------+------+----+------+------+-----------------+---------------+
|    9|  0.34|   0.255| 0.095|   1|     0|     0|            0.143|       8.482176|
|    8| 0.585|    0.45|  0.15|   0|     0|     1|           0.6385|       8.534326|
|    5| 0.255|    0.19|  0.07|   0|     0|     1|           0.0505|      5.6782026|
|    9| 0.565|    0.44| 0.135|   0|     0|     1|           0.5205|       8.723779|
|   10| 0.575|    0.45| 0.135|   0|     1|     0|           0.5925|       10.79155|
+-----+------+--------+------+----+------+------+-----------------+---------------+
only showing top 5 rows

Now that we have our results, we need to quantify our model's performance. We will use root mean square error (RMSE) to measure how close Estimated_rings is to the actual Rings value.

RMSE is a popular way to measure how closely a regression model predicts a response. A lower RMSE indicates a closer prediction.

Here is the equation for RMSE:

\begin{equation*}
RMSE = \sqrt{\frac{\sum_{i=1}^n (\hat{y_i}-y_i)^2}{N}}
\end{equation*}

where $\hat{y_i}$ is the number of predicted rings, $y_i$ is the observed number of rings, and N is the number of rows in the test data set.

We'll use Spark SQL to run a SQL query on our data to calculate the RMSE.

In [20]:
rings = inference_data.schema.names[0]
predicted_rings = inference_data.schema.names[-1]
table_name = 'inference'

inference_data.registerTempTable(table_name)
sql_rmse = 'SELECT SQRT(AVG(POWER({}-{}, 2))) AS RMSE FROM {}'.format(rings, predicted_rings, table_name)

rmse_results = spark.sql(sql_rmse)
rmse_results.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|             RMSE|
+-----------------+
|2.810150564902427|
+-----------------+

## Wrap-Up
Congratulations! You processed data in Apache Spark on EMR and trained and deployed a machine learning model in Amazon SageMaker! Feel free to try different combinations of models and hyperparameters to see if you can reduce your model's RMSE.